# 运输问题
2020011054金大为

## 伏格尔法
（1）导入需要的包

In [2]:
import numpy as np
import pandas as pd

（2）从excel导入数据

In [3]:
I=4#行数
C=6#列数
M=1000000#大M
filename='运输问题.xlsx'
data=pd.read_excel(filename)
data.rename({I:'销量'},inplace=True)
data3=data.copy()
data3.iloc[0:I,0:C]=0
data

,甲,乙,丙,丁,戊,己,产量
0,10,20,5,9,10,0,5
1,2,10,8,30,6,0,6
2,1,20,7,10,4,0,2
3,8,6,3,7,5,0,9
销量,4,4,6,2,4,2,22


（3）算差额

In [4]:
data.iloc[I,C]=0
data0=data.copy()
data1=data.copy()
data2=data.copy()
values=data1.iloc[0:I,0:C].values
values.sort()#通过排序求最小值和次小值的差
data2.iloc[0:I,C]=values[:,1]-values[:,0]
values=data0.iloc[0:I,0:C].values
values.sort(axis=0)
data2.iloc[I,0:C]=values[1,:]-values[0,:]
data2.rename(columns={'产量':'差额'},inplace=True)
data2.rename({'销量':'差额'},inplace=True)
data2

,甲,乙,丙,丁,戊,己,差额
0,10,20,5,9,10,0,5
1,2,10,8,30,6,0,2
2,1,20,7,10,4,0,1
3,8,6,3,7,5,0,3
差额,1,4,2,2,1,0,0


（4）求最大差额列的最小元素所在坐标

In [5]:
max1=data2.iloc[4].max()
max2=data2.iloc[:,C].max()
if max1>max2:
    s=data2.iloc[I].argmax()
    i=data2.iloc[0:I,s].argmin()
else:
    i=data2.iloc[:,C].argmax()
    s=data2.iloc[i,0:C].argmin()
print(i,s)

0 5


（5）作运输方案表

In [6]:
if data.iloc[I,s]<=data.iloc[i,C]:#找最小差价在行还是列
    data3.iloc[i,s]=data.iloc[I,s]#运输方案表内赋值
    data.iloc[i,C]-=data.iloc[I,s]
    data.iloc[:,s]=M#用大M替换划掉的列
else:
    data3.iloc[i,s]=data.iloc[i,C]
    data.iloc[I,s]-=data.iloc[i,C]
    data.iloc[i,:]=M
data

,甲,乙,丙,丁,戊,己,产量
0,10,20,5,9,10,1000000,3
1,2,10,8,30,6,1000000,6
2,1,20,7,10,4,1000000,2
3,8,6,3,7,5,1000000,9
销量,4,4,6,2,4,1000000,0


（6）第一次求出了运输方案数第一个元素

In [7]:
data3.iloc[:,:]=data3.values.astype(int)
data3

,甲,乙,丙,丁,戊,己,产量
0,0,0,0,0,0,2,5
1,0,0,0,0,0,0,6
2,0,0,0,0,0,0,2
3,0,0,0,0,0,0,9
销量,4,4,6,2,4,2,22


PS：（改进方法）大M在多数情况是可以用的,但是会有最大值超出大M的风险，后来又想到可以用NaN来替代，如下：

In [8]:
if data.iloc[I,s]<=data.iloc[i,C]:
    data3.iloc[i,s]=data.iloc[I,s]
    data.iloc[i,C]-=data.iloc[I,s]
    data.iloc[:,s]=np.nan
else:
    data3.iloc[i,s]=data.iloc[i,C]
    data.iloc[I,s]-=data.iloc[i,C]
    data.iloc[i,:]=np.nan
data

,甲,乙,丙,丁,戊,己,产量
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,10.0,8.0,30.0,6.0,1000000.0,6.0
2,1.0,20.0,7.0,10.0,4.0,1000000.0,2.0
3,8.0,6.0,3.0,7.0,5.0,1000000.0,9.0
销量,4.0,4.0,6.0,2.0,4.0,999997.0,0.0


（7）按前面的方法用for循环算出整个表

In [9]:
filename='运输问题.xlsx'
data=pd.read_excel(filename)
data.rename({I:'销量'},inplace=True)
data3=data.copy()
data3.iloc[0:I,0:C]=0
for i in range(I+C-2):
    data.iloc[I,C]=0
    data0=data.copy()
    data1=data.copy()
    data2=data.copy()
    values=data1.iloc[0:I,0:C].values
    values.sort()
    data2.iloc[0:I,C]=values[:,1]-values[:,0]
    values=data0.iloc[0:I,0:C].values
    values.sort(axis=0)
    data2.iloc[I,0:C]=values[1,:]-values[0,:]
    data2.rename(columns={'产量':'差额'},inplace=True)
    data2.rename({'销量':'差额'},inplace=True)
    max1=data2.iloc[4].max()
    max2=data2.iloc[:,C].max()
    if max1>max2:
        s=data2.iloc[I].argmax()
        i=data2.iloc[0:I,s].argmin()
    else:
        i=data2.iloc[:,C].argmax()
        s=data2.iloc[i,0:C].argmin()
    if data.iloc[I,s]<=data.iloc[i,C]:
        data3.iloc[i,s]=data.iloc[I,s]
        data.iloc[i,C]-=data.iloc[I,s]
        data.iloc[:,s]=np.nan
    else:
        data3.iloc[i,s]=data.iloc[i,C]
        data.iloc[I,s]-=data.iloc[i,C]
        data.iloc[i,:]=np.nan

伏格尔运输方案表如下

In [10]:
data3.iloc[:,:]=data3.values.astype(int)
data3

,甲,乙,丙,丁,戊,己,产量
0,0,0,3,0,0,2,5
1,4,0,0,0,2,0,6
2,0,0,0,0,2,0,2
3,0,4,3,2,0,0,9
销量,4,4,6,2,4,2,22


# 位势法求检验数

（1）更新data表格，求基向量单位运价表

In [11]:
data=pd.read_excel(filename)
data4=np.multiply(data3/data3,data)#将表中0替换为NaN
data4.iloc[I]=data4.iloc[:,C]=np.nan#将ui，vi替换成NaN
data4.iloc[0:2,C]=0#因为有退化问题先预设两个检验数
data4.rename(columns={'产量':'ui'},inplace=True)
data4.rename({'销量':'vi'},inplace=True)
data4

,甲,乙,丙,丁,戊,己,ui
0,NaN,NaN,5.0,NaN,NaN,0.0,0.0
1,2.0,NaN,NaN,NaN,6.0,NaN,0.0
2,NaN,NaN,NaN,NaN,4.0,NaN,NaN
3,NaN,6.0,3.0,7.0,NaN,NaN,NaN
vi,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data5=data4.notnull()#基向量设为True
data6=data4.isnull()#非基向量设为True
data5

,甲,乙,丙,丁,戊,己,ui
0,False,False,True,False,False,True,True
1,True,False,False,False,True,False,True
2,False,False,False,False,True,False,False
3,False,True,True,True,False,False,False
vi,False,False,False,False,False,False,False


（2）求基变量检验数

In [13]:
for n in range(I+C):
    data5=data4.notnull()
    for i in range(I):
        for c in range(C):
            if data5.iloc[i,c]:
                if data5.iloc[i,C]:
                    data4.iloc[I,c]=data4.iloc[i,c]-data4.iloc[i,C]
                if data5.iloc[I,c]:
                    data4.iloc[i,C]=data4.iloc[i,c]-data4.iloc[I,c]
data4

,甲,乙,丙,丁,戊,己,ui
0,NaN,NaN,5.0,NaN,NaN,0.0,0.0
1,2.0,NaN,NaN,NaN,6.0,NaN,0.0
2,NaN,NaN,NaN,NaN,4.0,NaN,-2.0
3,NaN,6.0,3.0,7.0,NaN,NaN,-2.0
vi,2.0,8.0,5.0,9.0,6.0,0.0,NaN


（3）补全检验数

In [14]:
for i in range(I):
    for c in range(C):
        if data6.iloc[i,c]:
            data4.iloc[i,c]=data.iloc[i,c]-data4.iloc[I,c]-data4.iloc[i,C]
data4

,甲,乙,丙,丁,戊,己,ui
0,8.0,12.0,5.0,0.0,4.0,0.0,0.0
1,2.0,2.0,3.0,21.0,6.0,0.0,0.0
2,1.0,14.0,4.0,3.0,4.0,2.0,-2.0
3,8.0,6.0,3.0,7.0,1.0,2.0,-2.0
vi,2.0,8.0,5.0,9.0,6.0,0.0,NaN


检验数都非负数，而非基变量检验数中有0，所以该问题有无穷解。

（4）求最优解

In [15]:
price=np.multiply(data3,data)
np.sum(price.iloc[0:I,0:C].values)

90

得到最优解为90,运输方案为data3

In [16]:
data3

,甲,乙,丙,丁,戊,己,产量
0,0,0,3,0,0,2,5
1,4,0,0,0,2,0,6
2,0,0,0,0,2,0,2
3,0,4,3,2,0,0,9
销量,4,4,6,2,4,2,22
